# Project 2 | ML | 2021-22
- João Santos, 76912
- João Carvalho, 106310

## Pneumonia Dataset
https://www.kaggle.com/datasets/artyomkolas/3-kinds-of-pneumonia

### Imports

In [ ]:
import datetime
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

print(tf.__version__)

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, recall_score

from tensorflow.keras import backend as K
from tensorflow.keras import layers
from tensorflow.keras import Model, Sequential
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Activation,Dropout,Conv2D, MaxPooling2D,BatchNormalization
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras import regularizers

import tensorflow_addons as tfa

### Setup Globals

In [ ]:
# Check: https://keras.io/examples/vision/image_classification_efficientnet_fine_tuning/

# MODEL = "CNN"
MODEL = "EfficientNet" 
efficientnet_model = tf.keras.applications.efficientnet.EfficientNetB0

BATCH_SIZE = 2
EPOCHS = 30
IM_SIZE_W = 224 
IM_SIZE_H = 224 
IM_SIZE_CH = 1 if MODEL == "CNN" else 3

AUTOTUNE = tf.data.experimental.AUTOTUNE

### Load images do Pandas DataFrame

In [ ]:
filenames = tf.io.gfile.glob('./Curated X-Ray Dataset/*/*')

data = pd.DataFrame()
for el in range(0, len(filenames)):
    target = filenames[el].split('/')[-2]
    path = filenames[el]
    
    data.loc[el, 'filename'] = path
    data.loc[el, 'class'] = target

print(data['class'].value_counts(dropna=False))
data

### Shuffle data and make class numerical

In [ ]:
data = shuffle(data)
data.reset_index(drop=True, inplace=True)

change = {
'Normal' : '0',
'Pneumonia-Bacterial': '1',
'Pneumonia-Viral' : '2',
'COVID-19' : '3',
}

data['class'] = data['class'].map(change)
data

### Split data

In [ ]:
# Split data into train and validation
train_data, val_data = train_test_split(data, test_size=0.1, stratify=data['class'])
print(train_data['class'].value_counts(dropna=False))
print(val_data['class'].value_counts(dropna=False))

# Split train into train and test
train_data, test_data = train_test_split(train_data, test_size=0.1, stratify=train_data['class'])
print(train_data['class'].value_counts(dropna=False))
print(test_data['class'].value_counts(dropna=False))

### Define ImageDataGenerator

In [ ]:
color_mode='grayscale' if MODEL == "CNN" else 'rgb'
scale = 1./255 if MODEL == "CNN" else 1

datagen = ImageDataGenerator(rescale = scale,
                            zoom_range= 0.05,
                            brightness_range=[0.9, 1.0],
                            height_shift_range=0.05, 
                            width_shift_range=0.05,
                            rotation_range=10, 
                            )

test_datagen = ImageDataGenerator(rescale = scale)

train_gen = datagen.flow_from_dataframe(train_data,
                                        x_col="filename",
                                        y_col="class",
                                        target_size=(IM_SIZE_W, IM_SIZE_H),
                                        color_mode=color_mode,
                                        batch_size=BATCH_SIZE,
                                        class_mode='categorical',
                                        shuffle=True,
                                        num_parallel_calls=AUTOTUNE)

val_gen = test_datagen.flow_from_dataframe(val_data,
                                        x_col="filename",
                                        y_col="class",
                                        target_size=(IM_SIZE_W, IM_SIZE_H),
                                        color_mode=color_mode,
                                        batch_size=BATCH_SIZE,
                                        class_mode='categorical',
                                        shuffle=False,
                                        num_parallel_calls=AUTOTUNE)

test_gen = test_datagen.flow_from_dataframe(test_data,
                                        x_col="filename",
                                        y_col="class",
                                        target_size=(IM_SIZE_W, IM_SIZE_H),
                                        color_mode=color_mode,
                                        batch_size=BATCH_SIZE,
                                        class_mode='categorical',
                                        shuffle=False,
                                        num_parallel_calls=AUTOTUNE)

### CNN Model

In [ ]:
def create_model_CNN():
    
    # Model input
    input_layer = layers.Input(shape=(IM_SIZE_W, IM_SIZE_H, IM_SIZE_CH), name='input')  

    # First block
    x = layers.Conv2D(filters=128, kernel_size=3, 
                        activation='relu', padding='same', 
                        name='conv2d_1')(input_layer)
    x = layers.MaxPool2D(pool_size=2, name='maxpool2d_1')(x)
    x = layers.Dropout(0.1, name='dropout_1')(x)

    # Second block
    x = layers.Conv2D(filters=128, kernel_size=3, 
                        activation='relu', padding='same', 
                        name='conv2d_2')(x)
    x = layers.MaxPool2D(pool_size=2, name='maxpool2d_2')(x)
    x = layers.Dropout(0.1, name='dropout_2')(x)

    # Third block
    x = layers.Conv2D(filters=128, kernel_size=3, 
                        activation='relu', padding='same', 
                        name='conv2d_3')(x)
    x = layers.MaxPool2D(pool_size=2, name='maxpool2d_3')(x)
    x = layers.Dropout(0.1, name='dropout_3')(x)

    # Fourth block
    x = layers.Conv2D(filters=256, kernel_size=3, 
                        activation='relu', padding='same', 
                        name='conv2d_4')(x)
    x = layers.MaxPool2D(pool_size=2, name='maxpool2d_4')(x)
    x = layers.Dropout(0.1, name='dropout_4')(x)

    # Fifth block
    x = layers.Conv2D(filters=256, kernel_size=3, 
                        activation='relu', padding='same', 
                        name='conv2d_5')(x)
    x = layers.MaxPool2D(pool_size=2, name='maxpool2d_5')(x)
    x = layers.Dropout(0.1, name='dropout_5')(x)

    # Sixth block
    x = layers.Conv2D(filters=512, kernel_size=3, 
                        activation='relu', padding='same', 
                        name='conv2d_6')(x)
    x = layers.MaxPool2D(pool_size=2, name='maxpool2d_6')(x)
    x = layers.Dropout(0.1, name='dropout_6')(x)

    # Seventh block
    x = layers.Conv2D(filters=512, kernel_size=3, 
                        activation='relu', padding='same', 
                        name='conv2d_7')(x)
    x = layers.MaxPool2D(pool_size=2, name='maxpool2d_7')(x)
    x = layers.Dropout(0.1, name='dropout_7')(x)
    
    # GlobalAveragePooling
    x = layers.GlobalAveragePooling2D(name='global_average_pooling2d')(x)   
    x = layers.Flatten()(x)
    
    # Head
    x = layers.Dense(1024,activation='relu')(x)
    x = layers.Dropout(0.1, name='dropout_head_2')(x)
    x = layers.Dense(128,activation='relu')(x)
    
    # Output
    output = layers.Dense(units=4, 
                            activation='softmax', 
                            name='output')(x)


    model = Model(input_layer, output)
        
    F_1_macro = tfa.metrics.f_scores.F1Score(num_classes=4, average="macro", name='f1_macro') 
    
    model.compile(optimizer='adam', 
                    loss='categorical_crossentropy', 
                    metrics=F_1_macro)

    return model

### EfficientNetB3 Model

In [ ]:
def create_model_B3(base_trainable = True):
    img_shape = (IM_SIZE_W, IM_SIZE_H, IM_SIZE_CH)

    base_model=efficientnet_model(include_top=False,
        weights="imagenet",
        input_shape=img_shape, 
        pooling='max') 

    base_model.trainable=base_trainable

    x=base_model.output
    x=BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(x)
    x = Dense(256, kernel_regularizer = regularizers.l2(l = 0.016),activity_regularizer=regularizers.l1(0.006),
                bias_regularizer=regularizers.l1(0.006) ,activation='relu')(x)

    x=Dropout(rate=.4)(x)       
    output=Dense(4, activation='softmax')(x)
    model=Model(inputs=base_model.input, outputs=output)

    lr=.001 # start with this learning rate
    model.compile(Adamax(learning_rate=lr), loss='categorical_crossentropy', metrics=['accuracy']) 

    return model

### Create Model

In [ ]:
model = create_model_CNN() if MODEL == "CNN" else create_model_B3(False)
model.summary()

### Train the model

In [ ]:
init_time = datetime.datetime.now()


train_steps = train_gen.samples // BATCH_SIZE
valid_steps = val_gen.samples // BATCH_SIZE

early_stopping = EarlyStopping(monitor="val_loss", patience=4, mode="min", min_delta=0.01)

learning_rate_reduction = ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=2, 
                                            min_lr=1e-7, verbose=1, mode="min")

checkpoint = ModelCheckpoint("loss-{val_loss:.4f}.h5", monitor="val_loss", verbose=0, 
                             save_best_only=True, save_weights_only=True, mode="min")

history = model.fit(
    train_gen,
    validation_data=val_gen,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    steps_per_epoch=train_steps,
    validation_steps=valid_steps,
    callbacks=[
                checkpoint, 
                early_stopping, 
                learning_rate_reduction],
    verbose=1,
    )

required_time = datetime.datetime.now() - init_time
print(f'\nRequired time:  {str(required_time)}\n')

### Graph

In [ ]:
history_df = pd.DataFrame(history.history)
history_df.loc[0:, ['loss', 'val_loss']].plot()
print("Minimum Validation Loss: {:0.4f}".format(history_df['val_loss'].min()));

### Test the model

In [ ]:
test_steps = test_gen.samples // BATCH_SIZE

test_loss, test_acc = model.evaluate(test_gen, steps=test_steps)
print('\naccuracy:', test_acc, 'loss: ',test_loss)

In [ ]:
predict = model.predict(test_gen, steps=test_steps)
y_hat = np.argmax(predict, axis=1)
y_hat[:20]

In [ ]:
test_labels_df = pd.DataFrame()
test_labels_df[['class']] = test_data[['class']]

change = {
'0' : 0,
'1' : 1,
'2' : 2,
'3' : 3,
}

test_labels_df['class'] = test_labels_df['class'].map(change)
test_labels_df = test_labels_df[ : test_steps*BATCH_SIZE]


y_test = np.array(test_labels_df['class'])
y_test[:20]

### Classification Report

In [ ]:
print(classification_report(y_test, y_hat), '\n')
cm = confusion_matrix(y_test, y_hat)
sns.heatmap(cm, annot=True, cmap="Blues", fmt='.0f', cbar=False)

### Save the model

In [ ]:
path = './model'

if (not os.path.exists(path)):
    os.mkdir(path)
    
model.save(path+'/')